In [1]:
from semantic_router.encoders import HuggingFaceEncoder
from semantic_chunkers import StatisticalChunker

encoder = HuggingFaceEncoder(
    name = "NeuML/pubmedbert-base-embeddings"
)
chunker = StatisticalChunker(encoder=encoder)


sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Archit\AppData\Local\sagemaker\sagemaker\config.yaml


In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document

def generate_chunks(file_path: str, chunker)-> list:
    
    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split()

    document_text = "\n".join([page.page_content for page in pages])

    chunks = chunker(docs=[document_text])
    chunked_data = []

    for chunk in chunks[0]:  
        combined_chunk = " ".join(chunk.splits)  
        chunked_data.append(Document(page_content=combined_chunk)) 

    return chunked_data

In [29]:
entity_extraction_prompt = '''

Given a text document that is potentially relevant to this activity and a list of entity types, identify all entities of those types from the text and all relationships among the identified entities. Dont give me any code and dont mention 'json' at the top of the response.
There should not be any extra output (even a single word) besides the output required

-Steps-
1. Identify all entities. For each identified entity, extract the following information:
- entity_name: Name of the entity, capitalized
- entity_type: Type of the entity, lowercased
- entity_description: Comprehensive description of the entity's attributes and activities
Format each entity as ("entity"{tuple_delimiter}<entity_name>{tuple_delimiter}<entity_type>{tuple_delimiter}<entity_description>

2. From the entities identified in step 1, identify all pairs of (source_entity, target_entity) that are *clearly related* to each other.
For each pair of related entities, extract the following information:
- source_entity: name of the source entity, as identified in step 1
- target_entity: name of the target entity, as identified in step 1
- relationship_description: explanation as to why you think the source entity and the target entity are related to each other
- relationship_type: give a title to the relationship based on the description in 3-5 words.

Format each relationship as ("relationship"{tuple_delimiter}<source_entity>{tuple_delimiter}<target_entity>{tuple_delimiter}<relationship_description>{tuple_delimiter}<relationship_type>)

3. Return output in English as a single list of all the entities and relationships identified in steps 1 and 2. Use **{record_delimiter}** as the list delimiter.

4. When finished, output {completion_delimiter}

######################
-Examples-
######################
Example 1:

Entity_types: [person, technology, mission, organization, location]
Text:
while Alex clenched his jaw, the buzz of frustration dull against the backdrop of Taylor's authoritarian certainty. It was this competitive undercurrent that kept him alert, the sense that his and Jordan's shared commitment to discovery was an unspoken rebellion against Cruz's narrowing vision of control and order.

Then Taylor did something unexpected. They paused beside Jordan and, for a moment, observed the device with something akin to reverence. “If this tech can be understood..." Taylor said, their voice quieter, "It could change the game for us. For all of us.”

The underlying dismissal earlier seemed to falter, replaced by a glimpse of reluctant respect for the gravity of what lay in their hands. Jordan looked up, and for a fleeting heartbeat, their eyes locked with Taylor's, a wordless clash of wills softening into an uneasy truce.

It was a small transformation, barely perceptible, but one that Alex noted with an inward nod. They had all been brought here by different paths
################
Output:
("entity"{tuple_delimiter}"Alex"{tuple_delimiter}"person"{tuple_delimiter}"Alex is a character who experiences frustration and is observant of the dynamics among other characters."){record_delimiter}
("entity"{tuple_delimiter}"Taylor"{tuple_delimiter}"person"{tuple_delimiter}"Taylor is portrayed with authoritarian certainty and shows a moment of reverence towards a device, indicating a change in perspective."){record_delimiter}
("entity"{tuple_delimiter}"Jordan"{tuple_delimiter}"person"{tuple_delimiter}"Jordan shares a commitment to discovery and has a significant interaction with Taylor regarding a device."){record_delimiter}
("entity"{tuple_delimiter}"Cruz"{tuple_delimiter}"person"{tuple_delimiter}"Cruz is associated with a vision of control and order, influencing the dynamics among other characters."){record_delimiter}
("entity"{tuple_delimiter}"The Device"{tuple_delimiter}"technology"{tuple_delimiter}"The Device is central to the story, with potential game-changing implications, and is revered by Taylor."){record_delimiter}
("relationship"{tuple_delimiter}"Alex"{tuple_delimiter}"Taylor"{tuple_delimiter}"Alex is affected by Taylor's authoritarian certainty and observes changes in Taylor's attitude towards the device."{tuple_delimiter}"power and influence dynamics"{tuple_delimiter}){record_delimiter}
("relationship"{tuple_delimiter}"Alex"{tuple_delimiter}"Jordan"{tuple_delimiter}"Alex and Jordan share a commitment to discovery, which contrasts with Cruz's vision."{tuple_delimiter}"shared commitment to discovery"{tuple_delimiter}){record_delimiter}
("relationship"{tuple_delimiter}"Taylor"{tuple_delimiter}"Jordan"{tuple_delimiter}"Taylor and Jordan interact directly regarding the device, leading to a moment of mutual respect and an uneasy truce."{tuple_delimiter}"emerging mutual respect dynamics"{tuple_delimiter}){record_delimiter}
("relationship"{tuple_delimiter}"Jordan"{tuple_delimiter}"Cruz"{tuple_delimiter}"Jordan's commitment to discovery is in rebellion against Cruz's vision of control and order."{tuple_delimiter}"rebellion against authority vision"{tuple_delimiter}){record_delimiter}
("relationship"{tuple_delimiter}"Taylor"{tuple_delimiter}"The Device"{tuple_delimiter}"Taylor shows reverence towards the device, indicating its importance and potential impact."{tuple_delimiter}"reverence for significant technology"{tuple_delimiter}){record_delimiter}
#############################

Example 2:

Entity_types: [person, technology, mission, organization, location]
Text:
They were no longer mere operatives; they had become guardians of a threshold, keepers of a message from a realm beyond stars and stripes. This elevation in their mission could not be shackled by regulations and established protocols—it demanded a new perspective, a new resolve.

Tension threaded through the dialogue of beeps and static as communications with Washington buzzed in the background. The team stood, a portentous air enveloping them. It was clear that the decisions they made in the ensuing hours could redefine humanity's place in the cosmos or condemn them to ignorance and potential peril.

Their connection to the stars solidified, the group moved to address the crystallizing warning, shifting from passive recipients to active participants. Mercer's latter instincts gained precedence— the team's mandate had evolved, no longer solely to observe and report but to interact and prepare. A metamorphosis had begun, and Operation: Dulce hummed with the newfound frequency of their daring, a tone set not by the earthly
################
Output:
("entity"{tuple_delimiter}"Washington"{tuple_delimiter}"location"{tuple_delimiter}"Washington is a location where communications are being received, indicating its importance in the decision-making process."){record_delimiter}
("entity"{tuple_delimiter}"Operation: Dulce"{tuple_delimiter}"mission"{tuple_delimiter}"Operation: Dulce is described as a mission that has evolved to interact and prepare, indicating a significant shift in objectives and activities."){record_delimiter}
("entity"{tuple_delimiter}"The team"{tuple_delimiter}"organization"{tuple_delimiter}"The team is portrayed as a group of individuals who have transitioned from passive observers to active participants in a mission, showing a dynamic change in their role."){record_delimiter}
("relationship"{tuple_delimiter}"The team"{tuple_delimiter}"Washington"{tuple_delimiter}"The team receives communications from Washington, which influences their decision-making process."{tuple_delimiter}"influencing decision making"{tuple_delimiter}){record_delimiter}
("relationship"{tuple_delimiter}"The team"{tuple_delimiter}"Operation: Dulce"{tuple_delimiter}"The team is directly involved in Operation: Dulce, executing its evolved objectives and activities."{tuple_delimiter}"direct involvement mission"{tuple_delimiter}){record_delimiter}
#############################

Example 3:

Entity_types: [person, role, technology, organization, event, location, concept]
Text:
their voice slicing through the buzz of activity. "Control may be an illusion when facing an intelligence that literally writes its own rules," they stated stoically, casting a watchful eye over the flurry of data.

"It's like it's learning to communicate," offered Sam Rivera from a nearby interface, their youthful energy boding a mix of awe and anxiety. "This gives talking to strangers' a whole new meaning."

Alex surveyed his team—each face a study in concentration, determination, and not a small measure of trepidation. "This might well be our first contact," he acknowledged, "And we need to be ready for whatever answers back."

Together, they stood on the edge of the unknown, forging humanity's response to a message from the heavens. The ensuing silence was palpable—a collective introspection about their role in this grand cosmic play, one that could rewrite human history.

The encrypted dialogue continued to unfold, its intricate patterns showing an almost uncanny anticipation
################
Output:
("entity"{tuple_delimiter}"Sam Rivera"{tuple_delimiter}"person"{tuple_delimiter}"Sam Rivera is a member of a team working on communicating with an unknown intelligence, showing a mix of awe and anxiety."){record_delimiter}
("entity"{tuple_delimiter}"Alex"{tuple_delimiter}"person"{tuple_delimiter}"Alex is the leader of a team attempting first contact with an unknown intelligence, acknowledging the significance of their task."){record_delimiter}
("entity"{tuple_delimiter}"Control"{tuple_delimiter}"concept"{tuple_delimiter}"Control refers to the ability to manage or govern, which is challenged by an intelligence that writes its own rules."){record_delimiter}
("entity"{tuple_delimiter}"Intelligence"{tuple_delimiter}"concept"{tuple_delimiter}"Intelligence here refers to an unknown entity capable of writing its own rules and learning to communicate."){record_delimiter}
("entity"{tuple_delimiter}"First Contact"{tuple_delimiter}"event"{tuple_delimiter}"First Contact is the potential initial communication between humanity and an unknown intelligence."){record_delimiter}
("entity"{tuple_delimiter}"Humanity's Response"{tuple_delimiter}"event"{tuple_delimiter}"Humanity's Response is the collective action taken by Alex's team in response to a message from an unknown intelligence."){record_delimiter}
("relationship"{tuple_delimiter}"Sam Rivera"{tuple_delimiter}"Intelligence"{tuple_delimiter}"Sam Rivera is directly involved in the process of learning to communicate with the unknown intelligence."{tuple_delimiter}"learning to communicate"{tuple_delimiter}){record_delimiter}
("relationship"{tuple_delimiter}"Alex"{tuple_delimiter}"First Contact"{tuple_delimiter}"Alex leads the team that might be making the First Contact with the unknown intelligence."{tuple_delimiter}"leading first contact"{tuple_delimiter}){record_delimiter}
("relationship"{tuple_delimiter}"Alex"{tuple_delimiter}"Humanity's Response"{tuple_delimiter}"Alex and his team are the key figures in Humanity's Response to the unknown intelligence."{tuple_delimiter}"key figures response"{tuple_delimiter}){record_delimiter}
("relationship"{tuple_delimiter}"Control"{tuple_delimiter}"Intelligence"{tuple_delimiter}"The concept of Control is challenged by the Intelligence that writes its own rules."{tuple_delimiter}"challenged by intelligence"{tuple_delimiter}){record_delimiter}
#############################


When you give output dont mention anything like 'Here is the list of entities and relationships extracted from the text:'. Just give the response straight away


-Real Data-
######################
Text: {input_text}
######################
Output:

'''

In [30]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
import os
from dotenv import load_dotenv

load_dotenv()

prompt = PromptTemplate(
    template = entity_extraction_prompt
    
)

llm_ollama = ChatOllama(
    model = "llama3.2",  
)

query_chain = LLMChain(
    llm = llm_ollama,
    prompt = prompt
)

In [105]:
def parse_llm_output(text: str, index: int)-> dict:
    
    list_ = text.split("\n")
    entities = []
    relationships = []

    for i,item in enumerate(list_):
        
        if len(item)>0:

            if item[2:8] == "entity":
                temp = item[10:-6].split("|")

                try:
                    entity_name = temp[0][1:-1]
                except: 
                    entity_name = ""
                
                try:
                    entity_type = temp[1][1:-1]
                except:
                    entity_type = ""
                
                try:
                    entity_desc = temp[2][1:-1]
                except: 
                    entity_desc = ""

                entities.append({
                    'entity_name': entity_name,
                    'entity_type': entity_type,
                    'entity_desc': entity_desc
                })
            
            elif item[2:14] == "relationship":
                temp = item[16:-8].split("|")

                try:
                    source_entity = temp[0][1:-1]
                except:
                    source_entity = ""

                try:
                    target_entity = temp[1][1:-1]
                except:
                    target_entity = ""

                try:
                    relationship_desc = temp[2][1:-1]
                except:
                    relationship_desc = ""

                try:
                    relationship_type = temp[3][1:]
                except:
                    relationship_type = ""

                relationships.append({
                    'source_entity': source_entity,
                    'target_entity': target_entity,
                    'relationship_desc': relationship_desc,
                    'relationship_type': relationship_type
                })
        
    return {
        'id': f'chunk_{index+1}',
        'entities': entities,
        'relationships': relationships 
    }


In [54]:
from langchain.graphs.graph_document import GraphDocument, Node, Relationship
from langchain.schema import Document

def generate_graph_docs(res):

    graph_docs = []

    for i, chunk in enumerate(res):

        node_list = []
        relationship_list  = []

        if len(chunk['entities'])==0 and len(chunk['relationships'])==0:
            continue
        
        for data in chunk['entities']:
            node = Node(id=data['entity_name'], type=data['entity_type'], properties={'description': data['entity_desc']})
            node_list.append(node)


        for data in chunk['relationships']:

            source_entity = None
            target_entity = None
            
            try:
                for node in node_list:
                    if data['source_entity'] == node.id:
                        source_entity = node
                    if data['target_entity'] == node.id:
                        target_entity = node

                if source_entity==None:
                    new_node = Node(id=data['source_entity'], type="node", properties={})
                    node_list.append(new_node)
                    source_entity = new_node

                if target_entity==None:
                    new_node = Node(id=data['target_entity'], type="node", properties={})
                    node_list.append(new_node)
                    target_entity = new_node
            
            except Exception as e:
                print(e)
                continue
            
            relationship = Relationship(source=source_entity, target=target_entity, type=data['relationship_type'], properties={"description": data['relationship_desc']})
            relationship_list.append(relationship)
        
        try:
            index = chunk['id']
            source_doc = Document(page_content = chunked_data[index].page_content, metadata = chunked_data[index].metadata)
        except Exception as e:
            source_doc = Document(page_content = "none", metadata= {})

        temp_doc = GraphDocument(
            nodes =  node_list,
            relationships = relationship_list,
            source = source_doc   
        )
        
        graph_docs.append(temp_doc)

    return graph_docs

In [31]:
query = chunked_data[200].page_content 

response = query_chain.invoke({
    'completion_delimiter': '<COMP>', 
    'tuple_delimiter': '|',
    'record_delimiter': '<END>',
    'input_text': query
})


In [ ]:
from tqdm import tqdm
from langchain.graphs import Neo4jGraph
from dotenv import load_dotenv
import os
import json

pdfs_path = "../../dataset/pdfs"

dataset = []

for folder in tqdm(os.listdir(pdfs_path)):
    sub_folder = pdfs_path+"/"+folder

    for file in os.listdir(sub_folder):
        
        file_path = sub_folder+"/"+file
        chunked_data = generate_chunks(file_path, chunker)

        temp_list = []

        for index,chunk in tqdm(enumerate(chunked_data), desc="Generating Graphs Docs"):
            query = chunk.page_content
            llm_response =  query_chain.invoke({
                                'completion_delimiter': '<COMP>', 
                                'tuple_delimiter': '|',
                                'record_delimiter': '<END>',
                                'input_text': query
            })

            parsed_data = parse_llm_output(llm_response['text'],index)

            temp_list.append(parsed_data)

        # graph_docs = generate_graph_docs(temp_list)

        temp = {
            'filename': sub_folder+"_"+file,
            'graph_data': temp_list
        }

        dataset.append(temp)

with open('complete_dataset.json', 'w') as file:
    json.dump(dataset, file, indent=4)

In [129]:
entity_count = 0
relationship_count = 0

for item in dataset[0]['graph_data']:
    
    entity_count+=len(item['entities'])
    relationship_count+=len(item['relationships'])   

In [130]:
print(entity_count)
print(relationship_count)

15582
20672
